# 24.1

Write a python program using SQLObject to lookup the scientific name for a user-supplied organism name.

In [1]:
from sqlobject import *
import os.path

In [2]:
db = 'small_taxa.db3'
conn_str = os.path.abspath(db)
conn = 'sqlite:' + conn_str

sqlhub.processConnection = connectionForURI(conn)

class Taxonomy(SQLObject):
    class sqlmeta:
        Name = 'tax_id'
        fromDatabase = True
    names = MultipleJoin('Name', joinColumn='tax_id')

class Name(SQLObject):
    class sqlmeta:
        fromDatabase = True
    taxa = ForeignKey('Taxonomy', dbName='tax_id')

In [3]:
Taxonomy._connection.debug = True
taxa_schema = Taxonomy.select()
list(taxa_schema)

 1/Select  :  SELECT taxonomy.id, taxonomy.taxid, taxonomy.scientific_name, taxonomy.rank, taxonomy.parent_id FROM taxonomy WHERE 1 = 1
 1/QueryR  :  SELECT taxonomy.id, taxonomy.taxid, taxonomy.scientific_name, taxonomy.rank, taxonomy.parent_id FROM taxonomy WHERE 1 = 1
 1/COMMIT  :  auto


[<Taxonomy 1 taxid=1 scientificName='root' rank='no rank' parentID=1>,
 <Taxonomy 2 taxid=2759 scientificName='Eukaryota' rank='superkingdom' parentID=393>,
 <Taxonomy 3 taxid=6072 scientificName='Eumetazoa' rank='no rank' parentID=183>,
 <Taxonomy 4 taxid=7711 scientificName='Chordata' rank='phylum' parentID=186>,
 <Taxonomy 5 taxid=7742 scientificName='Vertebrata' rank='no rank' parentID=319>,
 <Taxonomy 6 taxid=7776 scientificName='Gnathostomata' rank='superclass' parentID=5>,
 <Taxonomy 7 taxid=8287 scientificName='Sarcopterygii' rank='no rank' parentID=351>,
 <Taxonomy 8 taxid=9347 scientificName='Eutheria' rank='no rank' parentID=181>,
 <Taxonomy 9 taxid=9443 scientificName='Primates' rank='order' parentID=539>,
 <Taxonomy 10 taxid=9445 scientificName='Lemuridae' rank='family' parentID=582>,
 <Taxonomy 11 taxid=9446 scientificName='Lemur' rank='genus' parentID=10>,
 <Taxonomy 12 taxid=9447 scientificName='Lemur catta' rank='species' parentID=11>,
 <Taxonomy 13 taxid=9452 scientif

In [4]:
Name._connection.debug = True
name_schema = Name.select()
list(name_schema)

 1/Select  :  SELECT name.id, name.tax_id, name.taxonomy_id, name.name, name.name_class FROM name WHERE 1 = 1
 1/QueryR  :  SELECT name.id, name.tax_id, name.taxonomy_id, name.name, name.name_class FROM name WHERE 1 = 1


OperationalError: no such column: name.tax_id